In [1]:
## Read the data into a dataframe
import json
import pandas as pd
import numpy as np
import re
import string
import nltk


df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

In [2]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

In [3]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [4]:
X=X['title']+' '+X['text']

X = X.apply(clean_text)

In [5]:
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

In [6]:
clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

In [7]:
fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

In [8]:
fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

In [9]:
real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)

In [10]:
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
to_txt=x_train+x_test

엘모벡터 생성

In [13]:
import os
os.chdir('C:\\Users\\user\\Desktop\\bilm-tf-master')

In [14]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 
#사전 학습된 모델이 있는 디렉토리
datadir = os.path.join('swb', 'model(WELFake256)')
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
batcher = Batcher(vocab_file, 50)
 
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
context_embeddings_op = bilm(context_character_ids)
 
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)
 
#테스트 임베딩
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [15]:
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())     
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [16]:
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  10.67 
Dress-Fashion =  10.56


Cosine Distance Comparison - 

Dress-Technology =  0.66 
Dress-Fashion =  0.74


임베딩

In [17]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_lst=[]

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_train)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('C:/Users/user/Desktop/Multimodal TextCuboid/WELFake 분류/elmo_embedding/train(WELFake256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 317, 256)
Shape of generated embeddings =  (1, 13, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 423, 256)
Shape of generated embeddings =  (1, 457, 256)
Shape of generated embeddings =  (1, 645, 256)
Shape of generated embeddings =  (1, 339, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 278, 256)
Shape of generated embeddings =  (1, 840, 256)
Shape of generated embeddings =  (1, 1052, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 679, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 654, 256)
Shape of generated embeddings =  (1, 336, 256)
Shape of generated embeddings =  (1, 366, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of generated embeddings =  (1, 599, 256)
Shape of generat

Shape of generated embeddings =  (1, 272, 256)
Shape of generated embeddings =  (1, 1067, 256)
Shape of generated embeddings =  (1, 139, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 443, 256)
Shape of generated embeddings =  (1, 456, 256)
Shape of generated embeddings =  (1, 406, 256)
Shape of generated embeddings =  (1, 1031, 256)
Shape of generated embeddings =  (1, 347, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 97, 256)
Shape of generated embeddings =  (1, 907, 256)
Shape of generated embeddings =  (1, 422, 256)
Shape of generated embeddings =  (1, 223, 256)
Shape of generated embeddings =  (1, 1171, 256)
Shape of generated embeddings =  (1, 334, 256)
Shape of generated embeddings =  (1, 576, 256)
Shape of generated embeddings =  (1, 139, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 303, 256)
Shape of generated embeddings =  (1, 376, 256)
Shape of gene

Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 289, 256)
Shape of generated embeddings =  (1, 156, 256)
Shape of generated embeddings =  (1, 647, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 609, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 220, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 304, 256)
Shape of generated embeddings =  (1, 855, 256)
Shape of generated embeddings =  (1, 468, 256)
Shape of generated embeddings =  (1, 597, 256)
Shape of generated embeddings =  (1, 358, 256)
Shape of generated embeddings =  (1, 916, 256)
Shape of generated embeddings =  (1, 569, 256)
Shape of generated embeddings =  (1, 373, 256)
Shape of generated embeddings =  (1, 255, 256)
Shape of generated embeddings =  (1, 1359, 256)
Shape of generated embeddings =  (1, 71, 256)
Shape of generated embeddings =  (1, 688, 256)
Shape of generat

Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 948, 256)
Shape of generated embeddings =  (1, 231, 256)
Shape of generated embeddings =  (1, 140, 256)
Shape of generated embeddings =  (1, 606, 256)
Shape of generated embeddings =  (1, 769, 256)
Shape of generated embeddings =  (1, 220, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 467, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 1987, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embeddings =  (1, 270, 256)
Shape of generated embeddings =  (1, 386, 256)
Shape of generated embeddings =  (1, 82, 256)
Shape of generated embeddings =  (1, 309, 256)
Shape of generated embeddings =  (1, 342, 256)
Shape of generated embeddings =  (1, 356, 256)
Shape of generated embeddings =  (1, 433, 256)
Shape of generated embeddings =  (1, 787, 256)
Shape of generat

Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 482, 256)
Shape of generated embeddings =  (1, 226, 256)
Shape of generated embeddings =  (1, 563, 256)
Shape of generated embeddings =  (1, 220, 256)
Shape of generated embeddings =  (1, 626, 256)
Shape of generated embeddings =  (1, 201, 256)
Shape of generated embeddings =  (1, 412, 256)
Shape of generated embeddings =  (1, 505, 256)
Shape of generated embeddings =  (1, 699, 256)
Shape of generated embeddings =  (1, 384, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of generated embeddings =  (1, 258, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 406, 256)
Shape of generated embeddings =  (1, 200, 256)
Shape of generated embeddings =  (1, 264, 256)
Shape of generated embeddings =  (1, 344, 256)
Shape of generated embeddings =  (1, 230, 256)
Shape of generated embeddings =  (1, 516, 256)
Shape of generated embeddings =  (1, 1281, 256)
Shape of gener

...

Shape of generated embeddings =  (1, 526, 256)
Shape of generated embeddings =  (1, 338, 256)
Shape of generated embeddings =  (1, 1147, 256)
Shape of generated embeddings =  (1, 724, 256)
Shape of generated embeddings =  (1, 502, 256)
Shape of generated embeddings =  (1, 797, 256)
Shape of generated embeddings =  (1, 351, 256)
Shape of generated embeddings =  (1, 338, 256)
Shape of generated embeddings =  (1, 332, 256)
Shape of generated embeddings =  (1, 439, 256)
Shape of generated embeddings =  (1, 368, 256)
Shape of generated embeddings =  (1, 491, 256)
Shape of generated embeddings =  (1, 413, 256)
Shape of generated embeddings =  (1, 898, 256)
Shape of generated embeddings =  (1, 309, 256)
Shape of generated embeddings =  (1, 231, 256)
Shape of generated embeddings =  (1, 176, 256)
Shape of generated embeddings =  (1, 1515, 256)
Shape of generated embeddings =  (1, 1095, 256)
Shape of generated embeddings =  (1, 513, 256)
Shape of generated embeddings =  (1, 759, 256)
Shape of g

Shape of generated embeddings =  (1, 722, 256)
Shape of generated embeddings =  (1, 271, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 257, 256)
Shape of generated embeddings =  (1, 244, 256)
Shape of generated embeddings =  (1, 234, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 349, 256)
Shape of generated embeddings =  (1, 233, 256)
Shape of generated embeddings =  (1, 357, 256)
Shape of generated embeddings =  (1, 797, 256)
Shape of generated embeddings =  (1, 409, 256)
Shape of generated embeddings =  (1, 826, 256)
Shape of generated embeddings =  (1, 456, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 717, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 406, 256)
Shape of generated embeddings =  (1, 470, 256)
Shape of generated embeddings =  (1, 187, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of gener

Shape of generated embeddings =  (1, 817, 256)
Shape of generated embeddings =  (1, 352, 256)
Shape of generated embeddings =  (1, 295, 256)
Shape of generated embeddings =  (1, 320, 256)
Shape of generated embeddings =  (1, 270, 256)
Shape of generated embeddings =  (1, 435, 256)
Shape of generated embeddings =  (1, 943, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 420, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embeddings =  (1, 349, 256)
Shape of generated embeddings =  (1, 629, 256)
Shape of generated embeddings =  (1, 391, 256)
Shape of generated embeddings =  (1, 336, 256)
Shape of generated embeddings =  (1, 251, 256)
Shape of generated embeddings =  (1, 204, 256)
Shape of generated embeddings =  (1, 258, 256)
Shape of generated embeddings =  (1, 688, 256)
Shape of generated embeddings =  (1, 223, 256)
Shape of generat

Shape of generated embeddings =  (1, 159, 256)
Shape of generated embeddings =  (1, 650, 256)
Shape of generated embeddings =  (1, 1245, 256)
Shape of generated embeddings =  (1, 276, 256)
Shape of generated embeddings =  (1, 373, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 792, 256)
Shape of generated embeddings =  (1, 613, 256)
Shape of generated embeddings =  (1, 1930, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 332, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of generated embeddings =  (1, 474, 256)
Shape of generated embeddings =  (1, 938, 256)
Shape of generated embeddings =  (1, 490, 256)
Shape of generated embeddings =  (1, 354, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 236, 256)
Shape of generated embeddings =  (1, 764, 256)
Shape of generated embeddings =  (1, 585, 256)
Shape of generated embeddings =  (1, 646, 256)
Shape of gen

Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 319, 256)
Shape of generated embeddings =  (1, 598, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated embeddings =  (1, 284, 256)
Shape of generated embeddings =  (1, 897, 256)
Shape of generated embeddings =  (1, 814, 256)
Shape of generated embeddings =  (1, 1118, 256)
Shape of generated embeddings =  (1, 306, 256)
Shape of generated embeddings =  (1, 327, 256)
Shape of generated embeddings =  (1, 739, 256)
Shape of generated embeddings =  (1, 827, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 524, 256)
Shape of generated embeddings =  (1, 410, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 426, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 321, 256)
Shape of generated embeddings =  (1, 550, 256)
Shape of generated embeddings =  (1, 229, 256)
Shape of genera

In [19]:
except_lst

[]

In [17]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_test_lst=[]

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

for i in range(len(x_test)):
    try:
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('C:/Users/user/Desktop/Multimodal TextCuboid/WELFake 분류/elmo_embedding/test(WELFake256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 372, 256)
Shape of generated embeddings =  (1, 543, 256)
Shape of generated embeddings =  (1, 13, 256)
Shape of generated embeddings =  (1, 896, 256)
Shape of generated embeddings =  (1, 1062, 256)
Shape of generated embeddings =  (1, 638, 256)
Shape of generated embeddings =  (1, 160, 256)
Shape of generated embeddings =  (1, 328, 256)
Shape of generated embeddings =  (1, 364, 256)
Shape of generated embeddings =  (1, 1261, 256)
Shape of generated embeddings =  (1, 716, 256)
Shape of generated embeddings =  (1, 1369, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 879, 256)
Shape of generated embeddings =  (1, 283, 256)
Shape of generated embeddings =  (1, 754, 256)
Shape of generated embeddings =  (1, 824, 256)
Shape of generated embeddings =  (1, 380, 256)
Shape of generated embeddings =  (1, 483, 256)
Shape of generated embeddings =  (1, 713, 256)
Shape of generated embeddings =  (1, 440, 256)
Shape of ge

Shape of generated embeddings =  (1, 996, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 464, 256)
Shape of generated embeddings =  (1, 229, 256)
Shape of generated embeddings =  (1, 635, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 70, 256)
Shape of generated embeddings =  (1, 419, 256)
Shape of generated embeddings =  (1, 697, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 193, 256)
Shape of generated embeddings =  (1, 718, 256)
Shape of generated embeddings =  (1, 197, 256)
Shape of generated embeddings =  (1, 372, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 195, 256)
Shape of generated embeddings =  (1, 420, 256)
Shape of generated embeddings =  (1, 1256, 256)
Shape of generate

Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 698, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 11, 256)
Shape of generated embeddings =  (1, 459, 256)
Shape of generated embeddings =  (1, 488, 256)
Shape of generated embeddings =  (1, 666, 256)
Shape of generated embeddings =  (1, 344, 256)
Shape of generated embeddings =  (1, 973, 256)
Shape of generated embeddings =  (1, 148, 256)
Shape of generated embeddings =  (1, 408, 256)
Shape of generated embeddings =  (1, 1232, 256)
Shape of generated embeddings =  (1, 529, 256)
Shape of generated embeddings =  (1, 385, 256)
Shape of generated embeddings =  (1, 1171, 256)
Shape of generated embeddings =  (1, 1058, 256)
Shape of generated embeddings =  (1, 256, 256)
Shape of generated embeddings =  (1, 356, 256)
Shape of generated embeddings =  (1, 354, 256)
Shape of generated embeddings =  (1, 396, 256)
Shape of generated embeddings =  (1, 485, 256)
Shape of gen

Shape of generated embeddings =  (1, 391, 256)
Shape of generated embeddings =  (1, 397, 256)
Shape of generated embeddings =  (1, 627, 256)
Shape of generated embeddings =  (1, 416, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 999, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 487, 256)
Shape of generated embeddings =  (1, 122, 256)
Shape of generated embeddings =  (1, 1404, 256)
Shape of generated embeddings =  (1, 161, 256)
Shape of generated embeddings =  (1, 1206, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 1845, 256)
Shape of generated embeddings =  (1, 640, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 531, 256)
Shape of generated embeddings =  (1, 1377, 256)
Shape of generated embeddings =  (1, 418, 256)
Shape of generated embeddings =  (1, 722, 256)
Shape of gen

Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 1401, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 256, 256)
Shape of generated embeddings =  (1, 283, 256)
Shape of generated embeddings =  (1, 108, 256)
Shape of generated embeddings =  (1, 393, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 425, 256)
Shape of generated embeddings =  (1, 191, 256)
Shape of generated embeddings =  (1, 1534, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 841, 256)
Shape of generated embeddings =  (1, 196, 256)
Shape of generated embeddings =  (1, 1258, 256)
Shape of generated embeddings =  (1, 384, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 160, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 91, 256)
Shape of genera

...

Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 462, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 147, 256)
Shape of generated embeddings =  (1, 351, 256)
Shape of generated embeddings =  (1, 394, 256)
Shape of generated embeddings =  (1, 1003, 256)
Shape of generated embeddings =  (1, 316, 256)
Shape of generated embeddings =  (1, 331, 256)
Shape of generated embeddings =  (1, 509, 256)
Shape of generated embeddings =  (1, 885, 256)
Shape of generated embeddings =  (1, 524, 256)
Shape of generated embeddings =  (1, 444, 256)
Shape of generated embeddings =  (1, 374, 256)
Shape of generated embeddings =  (1, 228, 256)
Shape of generated embeddings =  (1, 383, 256)
Shape of generated embeddings =  (1, 368, 256)
Shape of generated embeddings =  (1, 548, 256)
Shape of generated embeddings =  (1, 762, 256)
Shape of generated embeddings =  (1, 326, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of gene

Shape of generated embeddings =  (1, 456, 256)
Shape of generated embeddings =  (1, 1856, 256)
Shape of generated embeddings =  (1, 212, 256)
Shape of generated embeddings =  (1, 1441, 256)
Shape of generated embeddings =  (1, 1292, 256)
Shape of generated embeddings =  (1, 512, 256)
Shape of generated embeddings =  (1, 188, 256)
Shape of generated embeddings =  (1, 430, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 246, 256)
Shape of generated embeddings =  (1, 120, 256)
Shape of generated embeddings =  (1, 397, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 1764, 256)
Shape of generated embeddings =  (1, 275, 256)
Shape of generated embeddings =  (1, 803, 256)
Shape of generated embeddings =  (1, 1130, 256)
Shape of generated embeddings =  (1, 435, 256)
Shape of generated embeddings =  (1, 252, 256)
Shape of generated embeddings =  (1, 559, 256)
Shape of generated embeddings =  (1, 786, 256)
Shape of

Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 833, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 191, 256)
Shape of generated embeddings =  (1, 118, 256)
Shape of generated embeddings =  (1, 884, 256)
Shape of generated embeddings =  (1, 1172, 256)
Shape of generated embeddings =  (1, 525, 256)
Shape of generated embeddings =  (1, 308, 256)
Shape of generated embeddings =  (1, 831, 256)
Shape of generated embeddings =  (1, 1056, 256)
Shape of generated embeddings =  (1, 1109, 256)
Shape of generated embeddings =  (1, 273, 256)
Shape of generated embeddings =  (1, 326, 256)
Shape of generated embeddings =  (1, 303, 256)
Shape of generated embeddings =  (1, 1100, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 498, 256)
Shape of generated embeddings =  (1, 196, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 490, 256)
Shape of ge

Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 425, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 444, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 680, 256)
Shape of generated embeddings =  (1, 1130, 256)
Shape of generated embeddings =  (1, 462, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 376, 256)
Shape of generated embeddings =  (1, 782, 256)
Shape of generated embeddings =  (1, 490, 256)
Shape of generated embeddings =  (1, 557, 256)
Shape of generated embeddings =  (1, 213, 256)
Shape of generated embeddings =  (1, 608, 256)
Shape of generated embeddings =  (1, 1400, 256)
Shape of generated embeddings =  (1, 603, 256)
Shape of generated embeddings =  (1, 318, 256)
Shape of generated embeddings =  (1, 827, 256)
Shape of generated embeddings =  (1, 310, 256)
Shape of generated embeddings =  (1, 367, 256)
Shape of gene

Shape of generated embeddings =  (1, 553, 256)
Shape of generated embeddings =  (1, 289, 256)
Shape of generated embeddings =  (1, 647, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 814, 256)
Shape of generated embeddings =  (1, 156, 256)
Shape of generated embeddings =  (1, 193, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embeddings =  (1, 1062, 256)
Shape of generated embeddings =  (1, 344, 256)
Shape of generated embeddings =  (1, 668, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 327, 256)
Shape of generated embeddings =  (1, 407, 256)
Shape of generated embeddings =  (1, 423, 256)
Shape of generated embeddings =  (1, 559, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 479, 256)
Shape of generated embeddings =  (1, 599, 256)
Shape of generated embeddings =  (1, 898, 256)
Shape of gener

In [19]:
except_test_lst

[]